In [1]:
%defaultDatasource jdbc:h2:file:./database/data.db

# Destruir tabelas

In [2]:
DROP TABLE IF EXISTS ABOVEHAPPY;
DROP TABLE IF EXISTS ABOVERURAL;
DROP TABLE IF EXISTS HAPPYRURAL;
DROP TABLE IF EXISTS ABOVEHAPPY;
DROP TABLE IF EXISTS ABOVEEXPEC;
DROP TABLE IF EXISTS HAPPYVIDA;
DROP TABLE IF EXISTS HAPPYVIDA;
DROP TABLE IF EXISTS ABOVEGDP;
DROP TABLE IF EXISTS ABOVEHAPPY;
DROP TABLE IF EXISTS AVGHAPPY;
DROP TABLE IF EXISTS COUNTRIES;
DROP TABLE IF EXISTS SUBREGIONS;
DROP TABLE IF EXISTS REGIONS;
DROP TABLE IF EXISTS UNESCO;
DROP TABLE IF EXISTS UNESCODESC;
DROP TABLE IF EXISTS WHR;
DROP TABLE IF EXISTS ILOSTAT;

# Carregar tabelas

In [3]:
DROP TABLE IF EXISTS COUNTRIES;
DROP TABLE IF EXISTS SUBREGIONS;
DROP TABLE IF EXISTS REGIONS;
DROP TABLE IF EXISTS UNESCO;
DROP TABLE IF EXISTS UNESCODESC;
DROP TABLE IF EXISTS WHR;
DROP TABLE IF EXISTS ILOSTAT;


CREATE TABLE REGIONS(
    REGION VARCHAR(30),
    PRIMARY KEY(REGION)
) AS SELECT * FROM  CSVREAD('./countries_processed/regions.csv');

CREATE TABLE SUBREGIONS(
    SUBREGION VARCHAR(50),
    REGION VARCHAR(30),
    PRIMARY KEY(SUBREGION),
    FOREIGN KEY(REGION)
        REFERENCES REGIONS(REGION)
            ON DELETE CASCADE
            ON UPDATE CASCADE
) AS SELECT * FROM  CSVREAD('./countries_processed/subregions.csv');

CREATE TABLE COUNTRIES(
    ALPHA3 VARCHAR(5),
    ALPHA2 VARCHAR(5),
    COUNTRY VARCHAR(100),
    SUBREGION VARCHAR(50),
    PRIMARY KEY(ALPHA3),
    FOREIGN KEY(SUBREGION)
        REFERENCES SUBREGIONS(SUBREGION)
            ON DELETE CASCADE
            ON UPDATE CASCADE
) AS SELECT 
    ALPHA3,
    ALPHA2,
    NAME,
    SUBREGION
FROM  CSVREAD('./countries_processed/countries.csv');


CREATE TABLE UNESCODESC(
    CODIGO VARCHAR(50),
    DESCRICAO VARCHAR(100),
    PRIMARY KEY(CODIGO)
) AS SELECT * FROM CSVREAD('./unesco_processed_data/unesco_descricao.csv');


CREATE TABLE UNESCO (
    Local VARCHAR(4) NOT NULL,
    ANO INTEGER,
    SP_DYN_LE00_IN DOUBLE ,
    SP_DYN_IMRT_IN DOUBLE,
    NY_GDP_MKTP_CD DOUBLE,
    NY_GDP_PCAP_CD DOUBLE,
    NY_GDP_MKTP_KD_ZG DOUBLE,
    "200343" DOUBLE,
    "200144" DOUBLE,
    "200345" DOUBLE,
    "200151" DOUBLE,
    SP_RUR_TOTL_ZS  DOUBLE,
    "200101" DOUBLE,
    
    PRIMARY KEY(Local, ANO) 

) AS SELECT
    Local,
    ANO,
    SP_DYN_LE00_IN,
    SP_DYN_IMRT_IN,
    NY_GDP_MKTP_CD,
    NY_GDP_PCAP_CD,
    NY_GDP_MKTP_KD_ZG,
    "200343",
    "200144",
    "200345",
    "200151",
    SP_RUR_TOTL_ZS,
    "200101"
FROM CSVREAD('./unesco_processed_data/unesco_interest_data.csv');


CREATE TABLE WHR(
    COUNTRYCODE VARCHAR(4),
    ANO INTEGER,
    HAPPINESSRANK INTEGER,
    HAPPINESSSCORE DOUBLE,
    GDPCAPITA DOUBLE,
    FAMILY DOUBLE,
    LIFEEXPECTANCY DOUBLE,
    FREEDOM DOUBLE,
    GOVCORRUPTION DOUBLE,
    GENEROSITY DOUBLE,
    
    PRIMARY KEY(COUNTRYCODE, ANO)    

) AS SELECT 
    ALPHA3,
    ANO,
    HAPPINESSRANK,
    HAPPINESSSCORE,
    GDPCAPITA,
    FAMILY,
    LIFEEXPECTANCY,
    FREEDOM,
    GOVCORRUPTION,
    GENEROSITY
FROM CSVREAD('./WHR_processed/whr_processado.csv');


CREATE TABLE ILOSTAT(
    LOCAL VARCHAR(4),
    ANO INTEGER,
    DESEMPREGO DOUBLE,
    POBREZA DOUBLE,
    
    PRIMARY KEY(LOCAL, ANO)    

) AS SELECT 
    LOCAL,
    ANO,
    DESEMPREGO,
    POBREZA
FROM CSVREAD('./ilostat_processed/ilostat_processed.csv');



In [4]:
SELECT * FROM REGIONS;
SELECT * FROM SUBREGIONS;
SELECT * FROM COUNTRIES;

In [5]:
SELECT * FROM UNESCO;
SELECT * FROM UNESCODESC;
SELECT * FROM WHR;
SELECT * FROM ILOSTAT;
-- ILOSTAT
-- populacao acima de 15 anos (%) abaixo da linha de probreza
-- população de 15 a 64 anos (%) desempregada

#### Dados da Unesco

-    "SP_DYN_LE00_IN", expectativa de vida
-    "SP_DYN_IMRT_IN", mortalidade infantil por 1000 nascimentos
-    "NY_GDP_MKTP_CD", GDP (current US$)
-    "NY_GDP_PCAP_CD", pib per capita
-    "NY_GDP_MKTP_KD_ZG",  GDP growth (annual %)
-    "200343", Population aged 14 years or younger (thousands)
-    "200144", Population aged 15-24 years (thousands)
-    "200345", Population aged 25-64 years (thousands) 
-    "200151", Population aged 65 years or older (thousands)
-    "SP_RUR_TOTL_ZS", Rural population (% of total population)
-    "200101"  Total population (thousands)

# Felicidade em 2016 vs GDP per capita

Analisando os dados de felicidade temos que:

- 50,34% dos países têm felicidade acima da média
- 76,19% dos países com GDP per capita acima da média são mais felizes
- Parece haver uma relação entre esses indicadores

In [6]:
-- FELICIDADE VS GDP PER CAPITA
DROP TABLE IF EXISTS ABOVEGDP;
DROP TABLE IF EXISTS ABOVEHAPPY;
DROP TABLE IF EXISTS AVGHAPPY;

-- MEDIA DE FELICIDADE 2016
CREATE VIEW AVGHAPPY AS
SELECT AVG(HAPPINESSSCORE) FROM WHR WHERE ANO = 2016;

-- PAISES COM FELICIDADE ACIMA DA MEDIA EM 2016
CREATE VIEW ABOVEHAPPY AS
SELECT COUNTRYCODE, HAPPINESSSCORE
    FROM WHR, AVGHAPPY A
    WHERE
        HAPPINESSSCORE > (SELECT * FROM AVGHAPPY) AND ANO = 2016;
        
-- PAISES COM GDP PER CAPITA ACIMA DA MEDIA
CREATE VIEW ABOVEGDP AS
SELECT COUNTRYCODE, GDPCAPITA
    FROM WHR
    WHERE
        GDPCAPITA > (SELECT AVG(GDPCAPITA) FROM WHR WHERE ANO = 2016) AND ANO = 2016;
        
        
-- TOTAL DE PAISES EM 2016
SELECT COUNT(COUNTRYCODE)
    FROM WHR
    WHERE ANO=2016;
    
-- TOTAL DE PAISES COM FELICIDADE ACIMA DA MEDIA EM 2016
SELECT COUNT(COUNTRYCODE)
    FROM ABOVEHAPPY;
    
-- TOTAL DE PAISES COM GDP PER CAPITA ACIMA DA MEDIA EM 2016
SELECT COUNT(COUNTRYCODE)
    FROM ABOVEGDP;
  
  
-- TOTAL DE PAISES COM FELICIDADE E GDP PER CAPITA ACIMA DE MEDIA EM 2016
SELECT COUNT(H.COUNTRYCODE)
    FROM ABOVEHAPPY H, ABOVEGDP G
    WHERE H.COUNTRYCODE = G.COUNTRYCODE;

# Felicidade vs expectativa de vida em 2016

Analisando os dados de felicidade temos que:

- Descartando países sem dados

- 50,00% dos países têm felicidade acima da média
- 74,41% dos países com expectativa de vida acima da média são mais felizes
- Parece haver uma relação entre esses indicadores

In [7]:
-- ANALISE FELICIDADE X Expectativa de vida"SP_DYN_LE00_IN" 
DROP TABLE IF EXISTS ABOVEHAPPY;
DROP TABLE IF EXISTS ABOVEEXPEC;
DROP TABLE IF EXISTS HAPPYVIDA;

CREATE VIEW HAPPYVIDA AS
SELECT W.COUNTRYCODE, W.ANO, W.HAPPINESSSCORE, U.SP_DYN_LE00_IN AS EXPECVIDA
    FROM WHR W, UNESCO U
    WHERE
        W.COUNTRYCODE = U.LOCAL AND W.ANO = U.ANO AND W.ANO = 2016;

CREATE VIEW ABOVEEXPEC AS
SELECT COUNTRYCODE, EXPECVIDA
    FROM HAPPYVIDA
    WHERE EXPECVIDA>(SELECT AVG(EXPECVIDA) FROM HAPPYVIDA)
    ORDER BY EXPECVIDA;
    
CREATE VIEW ABOVEHAPPY AS
SELECT COUNTRYCODE, HAPPINESSSCORE
    FROM HAPPYVIDA
    WHERE HAPPINESSSCORE>(SELECT AVG(HAPPINESSSCORE) FROM HAPPYVIDA)
    ORDER BY HAPPINESSSCORE;
    
-- TOTAL DE PAISES EM 2016
SELECT COUNT(COUNTRYCODE)
    FROM HAPPYVIDA;
    
-- TOTAL DE PAISES COM FELICIDADE ACIMA DA MEDIA EM 2016
SELECT COUNT(COUNTRYCODE)
    FROM ABOVEHAPPY;
    
-- TOTAL DE PAISES COM EXPECTATIVA DE VIDA ACIMA DA MEDIA EM 2016
SELECT COUNT(COUNTRYCODE)
    FROM ABOVEEXPEC;
  
  
-- TOTAL DE PAISES COM FELICIDADE E EXPECTATIVA DE VIDA ACIMA DE MEDIA EM 2016
SELECT COUNT(H.COUNTRYCODE)
    FROM ABOVEHAPPY H, ABOVEEXPEC E
    WHERE H.COUNTRYCODE = E.COUNTRYCODE;
    


# Felicidade vs população rural em 2016

Analisando os dados de felicidade temos que:

- Descartando países sem dados

- 48,95% dos países têm felicidade acima da média
- 24,29% dos países com população rural (%) acima da média são mais felizes
- Parece haver uma relação inversa entre esses indicadores

In [8]:
-- ANALISE FELICIDADE X PORCENTAGEM DE POPULAÇÃO RURAL (SP_RUR_TOTL_ZS)
DROP TABLE IF EXISTS ABOVEHAPPY;
DROP TABLE IF EXISTS ABOVERURAL;
DROP TABLE IF EXISTS HAPPYRURAL;

CREATE VIEW HAPPYRURAL AS
SELECT W.COUNTRYCODE, W.ANO, W.HAPPINESSSCORE, U.SP_RUR_TOTL_ZS AS PROPRURAL
    FROM WHR W, UNESCO U
    WHERE
        W.COUNTRYCODE = U.LOCAL AND W.ANO = U.ANO AND W.ANO = 2016
        AND U.SP_RUR_TOTL_ZS IS NOT NULL;
        

CREATE VIEW ABOVERURAL AS
SELECT COUNTRYCODE, PROPRURAL
    FROM HAPPYRURAL
    WHERE PROPRURAL>(SELECT AVG(PROPRURAL) FROM HAPPYRURAL)
    ORDER BY PROPRURAL;
    
CREATE VIEW ABOVEHAPPY AS
SELECT COUNTRYCODE, HAPPINESSSCORE
    FROM HAPPYRURAL
    WHERE HAPPINESSSCORE>(SELECT AVG(HAPPINESSSCORE) FROM HAPPYRURAL)
    ORDER BY HAPPINESSSCORE;
    
    
-- TOTAL DE PAISES EM 2016
SELECT COUNT(COUNTRYCODE)
    FROM HAPPYRURAL;
    
-- TOTAL DE PAISES COM FELICIDADE ACIMA DA MEDIA EM 2016
SELECT COUNT(COUNTRYCODE)
    FROM ABOVEHAPPY;
    
-- TOTAL DE PAISES COM POPULACAO RURAL (%) ACIMA DA MEDIA EM 2016
SELECT COUNT(COUNTRYCODE)
    FROM ABOVERURAL;
  
  
-- TOTAL DE PAISES COM FELICIDADE E EXPECTATIVA DE VIDA ACIMA DE MEDIA EM 2016
SELECT COUNT(H.COUNTRYCODE)
    FROM ABOVEHAPPY H, ABOVERURAL R
    WHERE H.COUNTRYCODE = R.COUNTRYCODE;
    



# Correlações

## 2016 ------------------------------------------

In [32]:
-- Droping tables if exists --
DROP TABLE IF EXISTS H_GDPPERCAPITA2016;
DROP TABLE IF EXISTS H_CORRUPTION2016;
DROP TABLE IF EXISTS H_FREEDOM2016;
DROP TABLE IF EXISTS H_GENEROSITY2016;
DROP TABLE IF EXISTS H_LIFEEXPECTANCY2016;
DROP TABLE IF EXISTS H_RURALPOP2016;
DROP TABLE IF EXISTS H_INFANCYMORTALITY2016;
DROP TABLE IF EXISTS H_BORNLIFE2016;
DROP TABLE IF EXISTS H_ACTUALCURRENCY2016;
DROP TABLE IF EXISTS H_DOLARGROWTH;
DROP TABLE IF EXISTS H_CHILDREN2016;
DROP TABLE IF EXISTS H_YOUTH2016;
DROP TABLE IF EXISTS H_ADULT2016;
DROP TABLE IF EXISTS H_OLD2016;
DROP TABLE IF EXISTS H_UNPLOYMENT2016;
DROP TABLE IF EXISTS H_POVERTY2016;

In [33]:
-- felicidade e gdppercapita
CREATE VIEW H_GDPPERCAPITA2016 AS
SELECT
    (AVG(W.HAPPINESSSCORE*W.GDPCAPITA) - (AVG(W.HAPPINESSSCORE)*AVG(W.GDPCAPITA)))/
    (STDDEV_POP(W.HAPPINESSSCORE) * STDDEV_POP(W.GDPCAPITA))
FROM WHR W
WHERE W.ANO=2016
;

In [34]:
-- felicidade e corrupcao
CREATE VIEW H_CORRUPTION2016 AS
SELECT
    (AVG(W.HAPPINESSSCORE*W.GOVCORRUPTION) - (AVG(W.HAPPINESSSCORE)*AVG(W.GOVCORRUPTION)))/
    (STDDEV_POP(W.HAPPINESSSCORE) * STDDEV_POP(W.GOVCORRUPTION))
FROM WHR W
WHERE W.ANO=2016
;

In [35]:
-- felicidade e liberdade
CREATE VIEW H_FREEDOM2016 AS
SELECT
    (AVG(W.HAPPINESSSCORE*W.FREEDOM) - (AVG(W.HAPPINESSSCORE)*AVG(W.FREEDOM)))/
    (STDDEV_POP(W.HAPPINESSSCORE) * STDDEV_POP(W.FREEDOM))
FROM WHR W
WHERE W.ANO=2016
;

In [36]:
-- felicidade e generosidade
CREATE VIEW H_GENEROSITY2016 AS
SELECT
    (AVG(W.HAPPINESSSCORE*W.GENEROSITY) - (AVG(W.HAPPINESSSCORE)*AVG(W.GENEROSITY)))/
    (STDDEV_POP(W.HAPPINESSSCORE) * STDDEV_POP(W.GENEROSITY))
FROM WHR W
WHERE W.ANO=2016
;

In [37]:
-- felicidade e expectativa de vida
CREATE VIEW H_LIFEEXPECTANCY2016 AS
SELECT
    (AVG(W.HAPPINESSSCORE*U.SP_DYN_LE00_IN) - (AVG(W.HAPPINESSSCORE)*AVG(U.SP_DYN_LE00_IN)))/
    (STDDEV_POP(W.HAPPINESSSCORE) * STDDEV_POP(U.SP_DYN_LE00_IN))
FROM WHR W, UNESCO U
WHERE W.ANO=2016 AND U.ANO=2016 AND W.COUNTRYCODE = U.LOCAL AND U.SP_DYN_LE00_IN IS NOT NULL
;

In [38]:
-- felicidade e populacao rural
CREATE VIEW H_RURALPOP2016 AS
SELECT
    (AVG(W.HAPPINESSSCORE*U.SP_RUR_TOTL_ZS) - (AVG(W.HAPPINESSSCORE)*AVG(U.SP_RUR_TOTL_ZS)))/
    (STDDEV_POP(W.HAPPINESSSCORE) * STDDEV_POP(U.SP_RUR_TOTL_ZS))
FROM WHR W, UNESCO U
WHERE W.ANO=2016 AND U.ANO=2016 AND W.COUNTRYCODE = U.LOCAL AND U.SP_RUR_TOTL_ZS IS NOT NULL
;

In [39]:
-- felicidade e indice de mortalidade infantil
CREATE VIEW H_INFANCYMORTALITY2016 AS
SELECT
    (AVG(W.HAPPINESSSCORE*U.SP_DYN_IMRT_IN) - (AVG(W.HAPPINESSSCORE)*AVG(U.SP_DYN_IMRT_IN)))/
    (STDDEV_POP(W.HAPPINESSSCORE) * STDDEV_POP(U.SP_DYN_IMRT_IN))
FROM WHR W, UNESCO U
WHERE W.ANO=2016 AND U.ANO=2016 AND W.COUNTRYCODE = U.LOCAL AND U.SP_DYN_IMRT_IN IS NOT NULL
;

In [40]:
-- felicidade e expectativa de vida ao nascimento
CREATE VIEW H_BORNLIFE2016 AS
SELECT
    (AVG(W.HAPPINESSSCORE*U.SP_DYN_LE00_IN) - (AVG(W.HAPPINESSSCORE)*AVG(U.SP_DYN_LE00_IN)))/
    (STDDEV_POP(W.HAPPINESSSCORE) * STDDEV_POP(U.SP_DYN_LE00_IN))
FROM WHR W, UNESCO U
WHERE W.ANO=2016 AND U.ANO=2016 AND W.COUNTRYCODE = U.LOCAL AND U.SP_DYN_LE00_IN IS NOT NULL
;

In [41]:
-- felicidade e dolar atual
CREATE VIEW H_ACTUALCURRENCY2016 AS
SELECT
    (AVG(W.HAPPINESSSCORE*U.NY_GDP_MKTP_CD) - (AVG(W.HAPPINESSSCORE)*AVG(U.NY_GDP_MKTP_CD)))/
    (STDDEV_POP(W.HAPPINESSSCORE) * STDDEV_POP(U.NY_GDP_MKTP_CD))
FROM WHR W, UNESCO U
WHERE W.ANO=2016 AND U.ANO=2016 AND W.COUNTRYCODE = U.LOCAL AND U.NY_GDP_MKTP_CD IS NOT NULL
;

In [42]:
-- felicidade e crescimento do dolar
CREATE VIEW H_DOLARGROWTH AS
SELECT
    (AVG(W.HAPPINESSSCORE*U.NY_GDP_MKTP_KD_ZG) - (AVG(W.HAPPINESSSCORE)*AVG(U.NY_GDP_MKTP_KD_ZG)))/
    (STDDEV_POP(W.HAPPINESSSCORE) * STDDEV_POP(U.NY_GDP_MKTP_KD_ZG))
FROM WHR W, UNESCO U
WHERE W.ANO=2016 AND U.ANO=2016 AND W.COUNTRYCODE = U.LOCAL AND U.NY_GDP_MKTP_KD_ZG IS NOT NULL
;

In [43]:
-- Felicidade e população com menos de 14 anos 
CREATE VIEW H_CHILDREN2016 AS
SELECT
    (AVG(W.HAPPINESSSCORE*U."200343") - (AVG(W.HAPPINESSSCORE)*AVG(U."200343")))/
    (STDDEV_POP(W.HAPPINESSSCORE) * STDDEV_POP(U."200343"))
FROM WHR W, UNESCO U
WHERE W.ANO=2016 AND U.ANO=2016 AND W.COUNTRYCODE = U.LOCAL AND U."200343" IS NOT NULL
;

In [44]:
-- Felicidade e população entre 15 e 24 anos
CREATE VIEW H_YOUTH2016 AS
SELECT
    (AVG(W.HAPPINESSSCORE*U."200144") - (AVG(W.HAPPINESSSCORE)*AVG(U."200144")))/
    (STDDEV_POP(W.HAPPINESSSCORE) * STDDEV_POP(U."200144"))
FROM WHR W, UNESCO U
WHERE W.ANO=2016 AND U.ANO=2016 AND W.COUNTRYCODE = U.LOCAL AND U."200144" IS NOT NULL
;

In [45]:
-- Felicidade e população entre 25 e 64 anos
CREATE VIEW H_ADULT2016 AS
SELECT
    (AVG(W.HAPPINESSSCORE*U."200345") - (AVG(W.HAPPINESSSCORE)*AVG(U."200345")))/
    (STDDEV_POP(W.HAPPINESSSCORE) * STDDEV_POP(U."200345"))
FROM WHR W, UNESCO U
WHERE W.ANO=2016 AND U.ANO=2016 AND W.COUNTRYCODE = U.LOCAL AND U."200345" IS NOT NULL
;

In [46]:
-- Felicidade e população com mais de 65 anos
CREATE VIEW H_OLD2016 AS
SELECT
    (AVG(W.HAPPINESSSCORE*U."200151") - (AVG(W.HAPPINESSSCORE)*AVG(U."200151")))/
    (STDDEV_POP(W.HAPPINESSSCORE) * STDDEV_POP(U."200151"))
FROM WHR W, UNESCO U
WHERE W.ANO=2016 AND U.ANO=2016 AND W.COUNTRYCODE = U.LOCAL AND U."200151" IS NOT NULL
;

In [47]:
-- felicidade e desemprego
CREATE VIEW H_UNPLOYMENT2016 AS
SELECT
    (AVG(W.HAPPINESSSCORE*I.DESEMPREGO) - (AVG(W.HAPPINESSSCORE)*AVG(I.DESEMPREGO)))/
    (STDDEV_POP(W.HAPPINESSSCORE) * STDDEV_POP(I.DESEMPREGO))
FROM WHR W, ILOSTAT I
WHERE W.ANO=2016 AND I.ANO=2016 AND W.COUNTRYCODE = I.LOCAL AND I.DESEMPREGO IS NOT NULL
;

In [48]:
-- felicidade e pobreza
CREATE VIEW H_POVERTY2016 AS
SELECT
    (AVG(W.HAPPINESSSCORE*I.POBREZA) - (AVG(W.HAPPINESSSCORE)*AVG(I.POBREZA)))/
    (STDDEV_POP(W.HAPPINESSSCORE) * STDDEV_POP(I.POBREZA))
FROM WHR W, ILOSTAT I
WHERE W.ANO=2016 AND I.ANO=2016 AND W.COUNTRYCODE = I.LOCAL AND I.POBREZA IS NOT NULL
;

## Tabela formada

In [55]:
-- TESTA ESSE, GABS --

SELECT H_GDPPERCAPITA2016.*, H_CORRUPTION2016.*, H_FREEDOM2016.*, H_GENEROSITY2016.*, H_LIFEEXPECTANCY2016.*, 
H_RURALPOP2016.*, H_INFANCYMORTALITY2016.*, H_BORNLIFE2016.*, H_ACTUALCURRENCY2016.*, H_DOLARGROWTH.*, H_CHILDREN2016.*, H_YOUTH2016.*, 
H_ADULT2016.*, H_OLD2016.*, H_UNPLOYMENT2016.*, H_POVERTY2016.*
FROM H_GDPPERCAPITA2016, H_CORRUPTION2016, H_FREEDOM2016, H_GENEROSITY2016, H_LIFEEXPECTANCY2016, 
H_RURALPOP2016, H_INFANCYMORTALITY2016, H_BORNLIFE2016, H_ACTUALCURRENCY2016, H_DOLARGROWTH, H_CHILDREN2016, H_YOUTH2016, 
H_ADULT2016, H_OLD2016, H_UNPLOYMENT2016, H_POVERTY2016;


# Função de semelhança

In [25]:
DROP TABLE IF EXISTS PAISINDICE;
DROP TABLE IF EXISTS NORMIND;


-- TABELA COM INDICADORES NORMALIZADOS

CREATE VIEW NORMIND AS
SELECT 
    U.LOCAL, 
    (U.SP_RUR_TOTL_ZS/(SELECT MAX(U.SP_RUR_TOTL_ZS) FROM UNESCO U)) AS PROPRURAL, 
    (U.SP_DYN_LE00_IN/(SELECT MAX(U.SP_DYN_LE00_IN) FROM UNESCO U)) AS EXPECVIDA,
    (W.GDPCAPITA/(SELECT MAX(W.GDPCAPITA) FROM WHR W)) AS GDPCAPITA
FROM UNESCO U, WHR W
WHERE 
    U.LOCAL = W.COUNTRYCODE AND U.ANO = 2016 AND W.ANO = 2016 AND
    U.SP_RUR_TOTL_ZS IS NOT NULL AND
    U.SP_DYN_LE00_IN IS NOT NULL AND
    W.GDPCAPITA IS NOT NULL
;


In [26]:
DROP TABLE IF EXISTS PAISINDICE;

CREATE VIEW PAISINDICE AS
SELECT
    I.LOCAL,
    W.HAPPINESSSCORE,
    ( (- I.PROPRURAL + I.EXPECVIDA + I.GDPCAPITA)/(SELECT MAX(- I.PROPRURAL + I.EXPECVIDA + I.GDPCAPITA) FROM NORMIND I) ) AS INDICE
FROM
    NORMIND I, WHR W
WHERE
    I.LOCAL = W.COUNTRYCODE AND W.ANO = 2016
ORDER BY INDICE;

CALL CSVWRITE('./database/pais_indice.csv', 'SELECT * FROM PAISINDICE');

SELECT * FROM PAISINDICE;